<a href="https://colab.research.google.com/github/SlinkyShelf/3BodyProblemAI/blob/main/3BodyProblemAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python Packages

In [ ]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

# Creating DataBase

In [ ]:
import numpy as np
import random as rd
import math

# Dataset Variables
sampleCount = 1
objects = 2

gravityConstant = 1

timeScale = .05
timeRange = 1
frameRange = [100, 400]

positionRange = 100
velocityRange = 20

setMass = 1

samples = []

# Random functions
def normalRand():
  return (rd.random()-.5)*2

def randRange(_range):
  return _range[0] + rd.random() * (_range[1] - _range[0])

def simulate(sample):

  # Creating manipulatable objects
  objects1 = []
  objects2 = []
  for obj in range(sample.objects):
    objects1.append(obj.copy())
    objects2.append(obj.copy())

  # Actual simulation
  for i in range(sample.frames):
    
    
    # Swapping Buffers
    temp = objects1
    objects1 = objects2
    objects2 = temp

  # End state
  sample.endState = objects1

def createSample():
  newsample = {}
  newsample.objects = []
  newsample.frames = randRange(frameRange)
  for i in range(objects):
    newobject = {}
    newobject.x = rd.random()*positionRange
    newobject.y = rd.random()*positionRange

    newobject.vx = normalRand()*velocityRange
    newobject.vy = normalRand()*velocityRange

    newobject.mass = setMass
    newsample.objects.append(newobject)

  simulate(newsample)

  return newsample

for i in range(sampleCount):
  samples.append(createSample())


mkdir: cannot create directory ‘trainingdata’: File exists


# Creating the AI